In [ ]:
from nbtemplate import display_header, display_codetoggle, get_path
display_header('Performanceplots.ipynb', status='draft')

```
Where have all the photons gone, long time integrating?
Where have all the photons gone, long time ago?
Where have all the photons gone?
Obscuring structures picked them one by one.
```

## Goal

In the early phases of the planning the Arcus project predicted effective areas based on a spread sheet that simply took the geometric opening area of the SPOs and multiplied a bunch of numerical factors between 0 and 1  with that number, where most of these factors depend on energy. For example, the reflectivity of the SPOs can be approximated as one number for each energy assuming some "average" reflection angle.

A full ray-trace calculation can provide a more detailed prediction of the effective area, in particular in treating geometrical obscuration effects such as "How many photons hit the support structure of the CAT grating petal?" or "How many photons are lost in a chip gap between CCDs?" However, it is still useful to compare results with the analytic predictions to check the correctness of the ray-trace and to understand where the photons are lost.

## Caveats

It's surprisingly hard get multiplicative factors that say "How many photons are lost at this element" from the results of the ray-trace simulations. MARXS tracks the survival probability for each photon and thus photons carry a different weight. So, we can't just say "$x$ photons out of $N$ photons hit the CAT support structure and are absorbed, thus the multiplication factor is $x/N$". In the simulation interactions don't happen in the same sequence that an analytical calculation would use, e.g. in the simulation we loop over all gratings and for each grating we find the intersecting photons and then treat those (pick diffraction order, apply grating efficiency, etc.). At the end, we assign a probability=0 for all those photons that did not hit any grating, assuming that all those will be absorbed by the structure of the CAT petal. However, at that point all photons that DO hit a grating already have adjusted probabilities from the grating efficiency etc.. Thus, we can't just naively take the sum of all probabilities before and after to get the fraction that is absorbed by the CAT petal structure.

The underlying problem is that in the simulations rays are additive and we can treat each ray on it's own, one after the other. However, when trying to get these multiplicative factors like "the CAT support petal absorbs x% of the photons" we need to compare all photons in the list and ensure that all rays are at the same step. Sometimes, that's not really possible. After all, we do a ray-trace to take into account effects that depend on the specific x,y,z location of an interaction.

In [ ]:
import numpy as np
from astropy.table import Table

import marxs
from marxs.simulator import KeepCol
import arcus
import arcus.arcus
from arcus.arcus import Arcus
from arcus.defaults import DefaultSource, DefaultPointing

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.sankey import Sankey

%matplotlib inline

In [ ]:
def detector_miss(photons):
    misses = ~np.isfinite(photons['det_x'])
    photons['probability'][misses] = 0
    return photons

In [ ]:
def get_probability_tables_from_simulations(energy, n_photons=2e4):
    mypointing = DefaultPointing()
    mysource = DefaultSource(energy=energy)
    photons = mysource.generate_photons(n_photons)
    photons = mypointing(photons)
    
    instrum = Arcus()
    keepprob = KeepCol('probability')
    keepprobspo = KeepCol('probability')
    keepprobcat = KeepCol('probability')
    keepprob(photons)
    instrum.postprocess_steps = [keepprob]
    instrum.elements[1].postprocess_steps = [keepprobspo]
    instrum.elements[2].postprocess_steps = [keepprobcat]
    instrum.elements.append(detector_miss)
    photons = instrum(photons)
    
    return keepprob.data, keepprobspo.data, keepprobcat.data

In [ ]:
from collections import OrderedDict

def multiplicativetable(p, pspo, pcat):
    '''Be weary of changes in the definition of the Arcus object.
    There is a lot of stuff hardcoded here.'''
    d = OrderedDict()  # can just be dict in 3.7 which guarantees dict ordering
    noSPO = (pspo[8] == 0) & (pspo[7] > 0)
    #d['XOU coverage in aperture rectangle'] = sum(pspo[8]) / sum(pspo[7])
    d['XOU coverage in aperture rectangle'] = noSPO.sum(dtype=float) / len(noSPO)
    for i in [p[1], pspo[7], pspo[8], pspo[9]]:
        i[noSPO] = 0
    d['SPO Reflectivity'] = sum(pspo[7]) / sum(p[1])
    d['SPO Geometry: Ribs and bars'] =  sum(pspo[9]) / sum(pspo[8])
    # Should be the multiplication of the two above. 
    d['SPO total'] = sum(p[2]) / sum(p[1])
    assert np.isclose(d['SPO Geometry: Ribs and bars'] * d['SPO Reflectivity'], d['SPO total'])
    noCAT = (pcat[5] == 0) & (pcat[4] > 0)
    catin = np.array(p[2], copy=True)
    catin[noCAT] = 0
    d['photons missing CAT grating'] = sum(catin) / sum(p[2])
    for i in pcat:
        i[noCAT] = 0
    d['CAT efficiency (incl. L1)'] = sum(pcat[3]) / sum(catin)
    d['CAT L2 support'] =  sum(pcat[4]) / sum(pcat[3])
    
    d['CAT Debye-Waller Factor'] =  sum(pcat[6]) / sum(pcat[5])
    d['CAT petal total'] =  sum(p[3]) / sum(p[2])
    assert np.isclose(d['CAT petal total'], 
                      d['CAT Debye-Waller Factor'] * d['photons missing CAT grating'] * 
                      d['CAT L2 support'] * d['CAT efficiency (incl. L1)'])
    d['Filter and CCD QE'] = sum(p[4]) / sum(p[3])
    d['Boom'] = sum(p[5]) / sum(p[4])
    d['Photons missing CCD'] = sum(p[-1]) / sum(p[6])
    return d

In [ ]:
tab025 = multiplicativetable(*get_probability_tables_from_simulations(.25))
tab05 = multiplicativetable(*get_probability_tables_from_simulations(.5))
tab10 = multiplicativetable(*get_probability_tables_from_simulations(1.0))
tab25 = multiplicativetable(*get_probability_tables_from_simulations(2.5))

In [ ]:
perffactors = Table([list(tab05.keys()), list(tab025.values()), list(tab05.values()),
                     list(tab10.values()), list(tab25.values())], 
                    names=['Factor', '0.25 keV', '0.5 keV', '1.0 keV', '2.5 keV'])

With the caveats given above, here is a table of multiplicative factors that shows which fraction of the total photon flux is lost in which step:

In [ ]:
for col in perffactors.colnames[1:]:
    perffactors[col].format='4.2f'
perffactors

Again, note that not all factors can be compared 1:1 to the analytic estimate. Notice that the Debye-Waller factors for the grating efficiency are much closer to 1 than you might expect. That's because this table includes 0 order photons. For e.g. 2.5 keV most photons end up in 0th order where the Debye-Waller factor is 1. Since we don't know at the beginning which photons go into which order it's not easy to sort out zeroth order photons.

It can be done (contact me if you have a significant need), but is outside of the scope of this document right now.

In [ ]:
'''For plotting porposes, I know multiply the factors above together, so I get an additative list of fluxes 
(not a multiplicative table) again. However, I want to do that in a particular order that differs from the order
in which these numbers are derived in the simulation, thus I do not use the probabilities arrays above directly.'''
d = tab05
base = {'flow factors': [1., d['SPO total'], d['CAT petal total'], d['Filter and CCD QE'], d['Boom'],
                         d['Photons missing CCD']],
        'labels': ['Input\nAperture', 'SPO', 'CAT petal', 'Filters\nand QE', 'boom', 'CCD misses', 'detected'],
        'orientations': [0, 1, 1, 1, 1, 1, 0],
}
spo = {'flow factors': [d['SPO Geometry: Ribs and bars'], d['SPO Reflectivity']],
      'labels': [None, 'Ribs and bars', 'Reflectivity'],
      'orientations': [-1, 1, 1],
      'scale': 1}
cat = {'flow factors': [d['photons missing CAT grating'], d['CAT efficiency (incl. L1)'],
                       d['CAT L2 support'], d['CAT Debye-Waller Factor']],
      'labels': [None, 'CAT petal\nstructure', 'efficiency\nand L1', 'L2', 'Debye-\nWaller'],
      'orientations': [-1, 1, 1, 1, 1],
      'scale': d['SPO total']}

for d in [base]:
    d['flow'] = np.hstack([[1], np.diff(np.cumprod(d['flow factors'])), [-np.prod(d['flow factors'])]])
    
for d in [spo, cat]:
    d['flow'] = d['scale'] * np.hstack([1. - np.prod(d['flow factors']), 
                           - (1 - np.array(d['flow factors'])) * 
                           np.cumprod(np.hstack([1, d['flow factors'][:-1]]))])

In [ ]:
def plot_sankey_photons_losses(base, spo, cat, title=''):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[],
                         title="Flow Diagram of a Widget")
    sankey = Sankey(ax=ax, scale=1,# offset=0.2, 
                    format='%.3f', unit='')
    sankey.add(flows=base['flow'],
               labels=base['labels'],
               orientations=base['orientations'],
               pathlengths=[.25, 1., 0.25, 0.25, 0.25, 0.6, 0.25],
               #patchlabel="Widget\nA"
              )
    sankey.add(flows=spo['flow'],
               orientations=spo['orientations'],
               labels=spo['labels'],
                prior=0,
              connect=(1, 0),
              label='SPOs',
            )
    sankey.add(flows=cat['flow'],
           orientations=cat['orientations'],
           labels=cat['labels'],
            prior=0,
          connect=(2, 0),
          label='CAT')
    diagrams = sankey.finish()
    diagrams[0].texts[-1].set_color('r')
    diagrams[0].text.set_fontweight('bold')
    ax.legend()
    ax.set_title(title)
    return fig, ax, diagrams

Below is a graphic representation of the total photon flow for 0.5 keV (about 24 Ang) photons. Note that the "detected photons" in the end again include both dispersed and zero order photons.

In [ ]:
fig, ax, diagrams = plot_sankey_photons_losses(base, spo, cat, '0.5 keV')

In [ ]:
display_codetoggle()